Goal:successfully run a hoomd.hpmc simulation of tetrahedra using the general polygon integration method 

In [1]:
import hoomd
import hoomd.hpmc
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline 

In [2]:
hoomd.context.initialize("--mode=cpu")

HOOMD-blue v2.6.0-8-g898f4c752 DOUBLE HPMC_MIXED MPI SSE SSE2 
Compiled: 06/06/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
-----
You are using HPMC. Please cite the following:
* J A Anderson, M E Irrgang, and S C Glotzer. "Scalable Metropolis Monte Carlo
  for simulation of hard shapes", Computer Physics Communications 204 (2016) 21
  --30
-----
HOOMD-blue is running on the CPU


In [110]:
16*16*30

7680

In [3]:
hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=3),n=int(7680**(1/3)))

notice(2): Group "all" created containing 6859 particles


In [4]:
#spheres
mc = hoomd.hpmc.integrate.sphere(seed=1, d=0.1, a=0.1)
mc.shape_param.set('A', diameter=1.0)

In [5]:
#tetrahedra
from utils import shapetools
from euclid.FreudShape import ConvexPolyhedron
hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=3),n=int(7680**(1/3)))
c_val = (2 * (1 - 0.59) + 1)
particle_vertices = shapetools.unitVolumeVerts(a=1,b=2,c=c_val)
freud_shape = ConvexPolyhedron(points=particle_vertices)
mc = hoomd.hpmc.integrate.convex_polyhedron(seed=1, d=0.1, a=0.1)
mc.shape_param.set('A', vertices=particle_vertices)
# mc.shape_param.set('A', vertices=[(1,0,-1/np.sqrt(2)), (-1,0,1/np.sqrt(2)),(0,-1,1/np.sqrt(2)),(0,+1,1/np.sqrt(2))])

In [6]:
all = hoomd.group.all()

In [96]:
d = hoomd.dump.gsd("tetrahedra.gsd", period=(10), group=all, overwrite=True)

In [103]:
d.dump_state(obj=mc)

In [104]:
free_vol = hoomd.hpmc.compute.free_volume(mc=mc, seed=123, nsample=100)

In [105]:
hoomd.analyze.log(filename="tetrahedra.log",
                 quantities=['hpmc_free_volume'],
                            period=(100),
                            overwrite=True)

In [7]:
for i in range(10):
    hoomd.run(30)

** starting run **
Time 00:00:08 | Step 30 / 30 | TPS 20.3609 | ETA 00:00:00
Average TPS: 20.3449
---------
notice(2): -- HPMC stats:
notice(2): Average translate acceptance: 0.999927
notice(2): Average rotate acceptance:    0.999971
notice(2): Trial moves per second:        557683
notice(2): Overlap checks per second:     1.19123e+07
notice(2): Overlap checks per trial move: 21.3603
notice(2): Number of overlap errors:      0
** run complete **
** starting run **
Time 00:00:09 | Step 60 / 60 | TPS 30.0098 | ETA 00:00:00
Average TPS: 29.8403
---------
notice(2): -- HPMC stats:
notice(2): Average translate acceptance: 0.998052
notice(2): Average rotate acceptance:    0.998558
notice(2): Trial moves per second:        816402
notice(2): Overlap checks per second:     1.86905e+07
notice(2): Overlap checks per trial move: 22.8938
notice(2): Number of overlap errors:      0
** run complete **
** starting run **
Time 00:00:10 | Step 90 / 90 | TPS 27.7305 | ETA 00:00:00
Average TPS: 27.7046
--

In [18]:
data = np.genfromtxt(fname='tetrahedra.log', skip_header=True);

In [19]:
plt.figure(figsize=(4,2.2), dpi=140);
plt.plot(data[:,0], data[:,1]);

IndexError: too many indices for array

<Figure size 560x308 with 0 Axes>

In [24]:
data

array([    0.  , 13906.25])